In [19]:
import pandas as pd
import os

In [12]:
base_dir = 'd:\\MyProject\\Rumor_Detection_Nikita'

In [13]:
os.listdir(base_dir)

['.idea', 'pre_process.py', 'run_model.py', 'tweets_file.csv']

In [20]:
df = pd.read_excel('D:\\MyProject\\Rumor_Detection_Nikita\\tweets_file_new.xlsx', encoding = 'utf-8')

In [21]:
df.head()

,Charlie Hebdo became well known for publishing the Muhammed cartoons two years ago,0
0,Charlie Hebdoâ€™s Last Tweet Before Shootings ...,0
1,Prediction: the #CharlieHebdo massacre will no...,0
2,10:28am Charlie Hebdo account mocks ISIS leade...,0
3,If your faith isn't strong enough to cope with...,0
4,Just arrived at scene of massacre #Paris #char...,0


In [22]:
df.columns = ['tweet', 'label']

In [23]:
df.head()

,tweet,label
0,Charlie Hebdoâ€™s Last Tweet Before Shootings ...,0
1,Prediction: the #CharlieHebdo massacre will no...,0
2,10:28am Charlie Hebdo account mocks ISIS leade...,0
3,If your faith isn't strong enough to cope with...,0
4,Just arrived at scene of massacre #Paris #char...,0


In [24]:
df = df.sample(frac=1).reset_index(drop=True)

In [47]:
import re
def preprocess(text):
    
    text = text.lower()
    text = re.sub('[-,.?~!$%&*/]','',text)
    
    return text
    
text = "hello this i$ some# sting/// with some .,m.m *& daf8 random things in it."
print(preprocess(text))

hello this i some# sting with some mm  daf8 random things in it


In [51]:
df_new = df.sample(frac=1).reset_index()
df_new

,index,tweet,label
0,89093,RT @BreakingNews: France's interior minister s...,1
1,14126,@verge @DaveLeeBBC Brilliant will private eye ...,0
2,18979,@TRobinsonNewEra @StarkRaving1 Exactly. https:...,0
3,3617,â€œ@mina_ysf: Religious leaders meet at Lakemb...,0
4,35558,@BBCBreaking Condemnation means nothing! Deman...,0
...,...,...,...
101840,23313,@tonyjdell @Mama4Obama1 @JeffersonObama worldwide,1
101841,66249,@decyberdiva everyone knows that white men ten...,1
101842,86978,@acidicliam I count three words in that pictur...,0
101843,33554,@AFPphoto @roncamma,1


In [48]:
df_new['tweet'].apply(preprocess)

49442           @daretoreadit @newday @cnn @camanpour hoax
81752    @ravithinkz i think the jokers at @theunrealti...
83428    @2afight there are little facts and all i was ...
23907    @skynews  wtf  descomunal eres enorme no te di...
1486     @ajcglobal l'amã©rique se tient avec la france...
                               ...                        
65280    @tarekfatah because they don't think he counts...
37585    @malengateacher @thevickyr thanks for clarifyi...
37661    @admiralmpj @eclark1st @groovesdc my son has f...
48852    @billydaleapologies but this is what i got\r\n...
28903    @julesaly @murdochsj @jeznews scott's claire d...
Name: tweet, Length: 101845, dtype: object

In [52]:
from sklearn.model_selection import train_test_split

x = df_new['tweet']
y = df_new['label']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.30, random_state = 42)

(30554,)

In [58]:
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# !wget --no-check-certificate \
#     https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
#     -O /tmp/sarcasm.json

vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000


# with open("/tmp/sarcasm.json", 'r') as f:
#     datastore = json.load(f)


# sentences = []
# labels = []
# urls = []
# for item in datastore:
#     sentences.append(item['headline'])
#     labels.append(item['is_sarcastic'])

# training_sentences = sentences[0:training_size]
# testing_sentences = sentences[training_size:]
# training_labels = labels[0:training_size]
# testing_labels = labels[training_size:]

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(x_train)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(x_train)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(x_test)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

num_epochs = 1
history = model.fit(training_padded, y_train, epochs=num_epochs, validation_data=(testing_padded, y_test), verbose=1)



Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 120, 16)           16000     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 64)                12544     
_________________________________________________________________
dense_4 (Dense)              (None, 24)                1560      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 25        
Total params: 30,129
Trainable params: 30,129
Non-trainable params: 0
_________________________________________________________________
Train on 71291 samples, validate on 30554 samples
71291/71291 [==============================] - 263s 4ms/sample - loss: 0.5176 - accuracy: 0.7568 - val_loss: 0.5031 - val_accuracy: 0.7695


TypeError: cannot convert the series to <class 'float'>

In [0]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

plot_graphs(history, 'acc')
plot_graphs(history, 'loss')

In [0]:
model.save("test.h5")